#Import Python Packages

In [5]:
import sys
sys.path.insert(0,'/global/homes/b/bpb/metatlas/metatlas/helpers/' )
import dill2plots as dp
%matplotlib inline

#Make blank file-info sheet

Get files from database

In [ ]:
files = metob.retrieve('LcmsRun',experiment='%experiment_name%',mzml_file='%', username='*')

# files = metob.retrieve('LcmsRun',mzml_file='%_LPSilva_Actino_%_51isolates%', username='*')
flist = []
for f in files:
    flist.append(f.mzml_file)
import numpy as np
flist = np.unique(flist)
print len(flist)
experiment_name = f.experiment

Save the files to an empty fileinfo sheet

In [ ]:
#dump all the files to a spreadheet, download it, and make a "filled in" one.
with open('aFileInfo_Sheet_%s_Not_filled_in.tab'%experiment_name,'w') as fid:
    fid.write('mzml_file\tgroup\tdescription\n')
    for f in flist:
        fid.write('%s\t\t\n'%f)

#Create metatlas groups from filled in file-info sheet

In [ ]:
df = pd.read_csv('../atlas_finfo_to_be_loaded/FileInfoSheet_AllFiles_20160413 (1).txt',sep='\t')
# df = pd.read_excel('../atlas_finfo_to_be_loaded/')
df

In [ ]:
grouped = df.groupby(by='group')
print grouped.groups.keys()

In [ ]:
for g in grouped.groups.keys():
    indices = grouped.groups[g]
    myGroup = metob.Group()
    myGroup.name = '%s'%g
    myGroup.description = df.loc[indices[0],'description']
    file_set = []
    for i in indices:
        file_set.append(metob.retrieve('LcmsRun',mzml_file='%%%s'%df.loc[i,'mzml_file'],username='*')[0])
    myGroup.items = file_set
    metob.store(myGroup)

#Create a new Atlas in metatlas from a spreadsheet

In [ ]:
df = pd.read_excel('../atlas_finfo_to_be_loaded/20160413_Atlas_HILIC_NEG (1).xlsx',sheetname='NEG Atlas')
df.columns = [x.lower() for x in df.columns]
df

In [ ]:

# compounds that have the wrong compound name will be listed
# Keep running this until no more compounds are listed
for x in df.index:
    if type(df.name[x]) != float or type(df.label[x]) != float:
        if type(df.name[x]) != float:
                if not metob.retrieve('Compounds',name=df.name[x]):
                    print df.name[x], "is not in database"

In [ ]:
myCompounds = []
rt_min = []
rt_max = []
rt_peak = []
mz = []
mz_tolerance = []
compound_label = []
for x in df.index:
    if type(df.name[x]) != float or type(df.label[x]) != float: #this logic is to skip empty rows
        if type(df.name[x]) != float: # this logic is where a name has been specified
            c = metob.retrieve('Compounds',name=df.name[x])[0] #currently, all copies of the molecule are returned.  The 0 is the most recent one. 
        else:
            c = 'use_label'
        if type(df.label[x]) != float:
            compound_label.append(df.label[x]) #if no name, then use label as descriptor
        else:
            compound_label.append('no label')
        myCompounds.append(c)
        rt_min.append(df.rt_min[x])
        rt_max.append(df.rt_max[x])
        rt_peak.append(df.rt_peak[x])
        mz.append(df.mz[x])
#     mz.append(c.mono_isotopic_molecular_weight + 1.007276)

        mz_tolerance.append(df.mz_threshold[x])
#     if abs(dmz) > 100:
#         dmz = 1e6 * (c.MonoIsotopic_molecular_weight + 59.013851 - df.mz[x] ) / df.mz[x]
#         print c.MonoIsotopic_molecular_weight - df.mz[x]
        try:
            dmz = 1e6 * (c.mono_isotopic_molecular_weight - 1.007276 - df.mz[x] ) / ( df.mz[x])
            print '%5.2f'%abs(dmz), "\t", c.name,c.formula,c.mono_isotopic_molecular_weight,c.description, df.rt_min[x], df.rt_max[x], df.mz[x]#, c.InChI  
        except:
            pass
    #TODO: See above todo's.  this block needs to be rebuilt
#     myID.description = 'mz=%5.4f,ppm=%5.4f,RTmin=%5.4f,RTmax=%5.4f,RTpeak=%5.4f'%(mz[i],
#                                                                                  mz_tolerance[i],
#                                                                                  rt_min[i],
#                                                                                  rt_max[i],
#                                                                                  rt_peak[i])
#     print myID.references


all_identifications = []
for i,c in enumerate(myCompounds):
    mzRef = metob.MzReference()

    # take the mz value from the spreadsheet
    mzRef.mz = mz[i]
    
    #TODO: calculate the mz from theoretical adduct and modification if provided.
#     mzRef.mz = c.MonoIso topic_molecular_weight + 1.007276
    mzRef.mz_tolerance = mz_tolerance[i]
    mzRef.mz_tolerance_units = 'ppm'
    mzRef.detected_polarity = 'negative'
#     mzRef.adduct = '[M-H]'
    
    rtRef = metob.RtReference()
    rtRef.rt_units = 'min'
    rtRef.rt_min = rt_min[i]
    rtRef.rt_max = rt_max[i]
    rtRef.rt_peak = rt_peak[i]
    
    myID = metob.CompoundIdentification()
    if c != 'use_label':
        myID.compound = [c]
    myID.name = compound_label[i]
    myID.mz_references = [mzRef]
    myID.rt_references = [rtRef]

    all_identifications.append(myID)

myAtlas = metob.Atlas()
#metob.Atlas() has "compound_identifications" and a "name"
# myAtlas.name = '20151130_LS_Negative_Hilic_QExactive_Archetypes'
myAtlas.name = 'RL_HM_6550_20160330_Neg_Hilic_Arkin_Ophelia'
myAtlas.compound_identifications = all_identifications
metob.store(myAtlas)

#Select groups of files to operate on

In [ ]:
temp_group = metob.retrieve('Groups', name = '20160413_Bhedlund_pHILIC_POS%', username='*')#RL_HM_6550_HILIC_20160328_%Neg%',username='*')
# temp_group = metob.retrieve('Groups', name = '%Replicate_Analysis_R2A_HILIC_neg%')
# print temp_group

group_filter = []

# group_filter = ['GBS_Med_Ctrl_Set1','GBS_Med_inc_Set1','JAD2_GBS_Set1',
#                 'GBS_4OE_Ctrl_Set1','JAD2_GBS_4OE_Set1','GBS_32OE_Ctrl_Set1',
#                 'GBS_32OE_Ctrl_Set2','GBS_32OE_inc_Set2','JAD2_GBS_32OE_Set1JAD2_GBS_32OE_Set2',
#                 'GBS_conc_inc_Set2']

if len(temp_group)==1:
    group = temp_group
else:
    group = []
    for i,g in enumerate(temp_group):
        if (len(g.items) > 0):# and ('RootCass' not in g.name) and ('_QC_' not in g.name):
            if group_filter:
                if any(ext in g.name for ext in group_filter):
                    group.append(g)
            else:
                group.append(g)

#Select Atlas to use

In [ ]:
atlas = metob.retrieve('Atlas',name = '%master%',username='*')
from datetime import datetime, date
for i,a in enumerate(atlas):
    print i, a.name,  datetime.utcfromtimestamp(a.last_modified)

In [ ]:
myAtlas = atlas[4]

#Get Data and Save it

In [ ]:
# get and pickle everything This is MSMS, raw MS1 datapoints, compound, group info, and file info
# combine positive and negative mode atlas, by join atlases together vs two compound references
# typically file-groups are NOT split by polarity, files with un-matched polarity will be discarded for analysis of an identification
data = dp.get_data_for_groups_and_atlas(group,myAtlas,output_filename)

#Load a pre-existing datafile

In [7]:
my_file = '/global/homes/b/bpb/notebooks/MetAtlas_Workflow_20151123/20160420_Bhedlund_pHILIC_NEG_AllGroups_master_atlas.pkl'
project_label = '20160420_Bhedlund_pHILIC_NEG_AllGroups_master_atlas'
# data = dp.get_data(my_file)

#Make Supplementary Tables

In [23]:

dp = reload(dp)

peak_height = dp.make_output_dataframe(input_fname = my_file, fieldname='peak_height' , output_loc=project_label+'/sheets')

#Error bar

In [28]:
dp = reload(dp)
dp.plot_errorbar_plots(peak_height,input_fname = my_file, output_loc=project_label+'/error_bar_peak_height')

#Make Chromatograms

In [9]:
dp = reload(dp)
dp.plot_all_compounds_for_each_file(input_fname = my_file, nCols = 12, scale_y=True , output_loc=project_label+'/all_compounds_each_file')
dp.plot_all_files_for_each_compound(input_fname = my_file, nCols = 12, scale_y=True , output_loc=project_label+'/all_files_each_compound')

7
74
nrows =  8
length of ymax is  5620


#Make Identification Figures

In [18]:
dp = reload(dp)
dp.make_identification_figure(input_fname = my_file, output_loc=project_label+'/identification')

#Export Atlas To Spreadsheet

In [ ]:
dp.export_atlas_to_spreadsheet(myAtlas,output_filename)
